# Using a Neural Network to Predict Bike Rentals

### Import Modules

In [90]:
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
#import statsmodels.formula.api
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.linear_model import LinearRegression

### Create Data Frame based on CSV

In [91]:
#bikes = pd.read_csv('bikeshare.csv', index_col='datetime',parse_dates = True)
bikes = pd.read_csv('bikeshare.csv')

bikes.head()
bikes

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,7,8
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8,6,14


### Create Training Data Frame

In [68]:
train_rows = int(len(bikes.index)*0.8)
test_rows = len(bikes.index)-train_rows

data_columns = ['datetime','season','holiday','workingday','weather','temp','atemp','humidity',
                'windspeed','casual','registered','count']

x_train = pd.DataFrame(columns = data_columns)
y_train = pd.DataFrame(columns = ['count'])

for i in range(train_rows):
    x_train.loc[i] = bikes.loc[i]

y_train = x_train.drop(columns = ['datetime','season','holiday','workingday','weather','temp','atemp','humidity',
                                  'windspeed','casual','registered'])
x_train = x_train.drop(columns = ['datetime', 'count'])
x_train

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered
0,1,0,0,1,9.84,14.395,81,0.0000,3,13
1,1,0,0,1,9.02,13.635,80,0.0000,8,32
2,1,0,0,1,9.02,13.635,80,0.0000,5,27
3,1,0,0,1,9.84,14.395,75,0.0000,3,10
4,1,0,0,1,9.84,14.395,75,0.0000,0,1
5,1,0,0,2,9.84,12.880,75,6.0032,0,1
6,1,0,0,1,9.02,13.635,80,0.0000,2,0
7,1,0,0,1,8.20,12.880,86,0.0000,1,2
8,1,0,0,1,9.84,14.395,75,0.0000,1,7
9,1,0,0,1,13.12,17.425,76,0.0000,8,6


### Create Testing Data Frame

In [67]:
x_test = pd.DataFrame(columns = data_columns)
y_test = pd.DataFrame(columns = ['count'])

for i in range(test_rows):
    x_test.loc[i] = bikes.loc[i + train_rows]

y_test = x_test.drop(columns = ['datetime','season','holiday','workingday','weather','temp','atemp','humidity',
                                'windspeed','casual','registered'])
x_test = x_test.drop(columns = ['datetime','count'])
x_test

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered
0,3,0,0,1,29.52,34.850,74,16.9979,1,18
1,3,0,0,1,29.52,34.850,79,16.9979,7,12
2,3,0,0,1,30.34,35.605,74,19.9995,18,50
3,3,0,0,1,31.16,36.365,66,22.0028,27,81
4,3,0,0,1,32.80,38.635,59,23.9994,61,168
5,3,0,0,1,32.80,38.635,59,27.9993,111,253
6,3,0,0,1,33.62,39.395,56,27.9993,155,282
7,3,0,0,1,35.26,40.150,47,35.0008,161,330
8,3,0,0,1,36.08,40.910,44,31.0009,208,315
9,3,0,0,1,36.90,42.425,42,31.0009,161,365


### Create Neural Network Structure

In [98]:
model = Sequential()
model.add(Dense(32, input_shape=(10,)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(optimizer='adam', loss='mse')

### Train Neural Network Structure

In [99]:
model.fit(x_train, y_train,
          epochs=100)

Epoch 1/100
8708/8708 [==============================] - 3s 312us/step - loss: 13503.9172
Epoch 2/100
8708/8708 [==============================] - 2s 251us/step - loss: 4077.6731
Epoch 3/100
8708/8708 [==============================] - 2s 251us/step - loss: 3486.5265
Epoch 4/100
8708/8708 [==============================] - 2s 250us/step - loss: 2967.3155
Epoch 5/100
8708/8708 [==============================] - 2s 256us/step - loss: 2916.5007
Epoch 6/100
8708/8708 [==============================] - 2s 250us/step - loss: 2820.0855
Epoch 7/100
8708/8708 [==============================] - 2s 251us/step - loss: 2602.3503
Epoch 8/100
8708/8708 [==============================] - 2s 259us/step - loss: 2510.8877
Epoch 9/100
8708/8708 [==============================] - 2s 254us/step - loss: 2501.4108
Epoch 10/100
8708/8708 [==============================] - 2s 256us/step - loss: 2345.8667
Epoch 11/100
8708/8708 [==============================] - 2s 251us/step - loss: 2376.3195
Epoch 12/100
8708/

Epoch 92/100
8708/8708 [==============================] - 2s 237us/step - loss: 822.6510
Epoch 93/100
8708/8708 [==============================] - 2s 227us/step - loss: 898.5556
Epoch 94/100
8708/8708 [==============================] - 2s 227us/step - loss: 810.7422
Epoch 95/100
8708/8708 [==============================] - 2s 234us/step - loss: 912.1045
Epoch 96/100
8708/8708 [==============================] - 2s 237us/step - loss: 783.4863
Epoch 97/100
8708/8708 [==============================] - 2s 240us/step - loss: 908.5154
Epoch 98/100
8708/8708 [==============================] - 2s 229us/step - loss: 748.5784
Epoch 99/100
8708/8708 [==============================] - 2s 233us/step - loss: 864.2879
Epoch 100/100
8708/8708 [==============================] - ETA: 0s - loss: 864.469 - 2s 233us/step - loss: 864.9545


### Predict and Analyze Network Accuracy using Testing Data Frame

In [100]:
prediction = model.predict(x_test)
prediction = np.round_(prediction)

actual = y_test['count']

fail_counter = 0


#print("Count prediction based on a dataset ")
for i in range(len(prediction)):
    # print("Predicted count: " + str(prediction[i]) + ' - Actual count:' + str(actual[i]) + ' - Correct Prediction: ' + str(prediction[i] == actual[i]))
    if (prediction[i] > (actual[i]+5)) or (prediction[i] < (actual[i]-5)):
        fail_counter += 1
    # print()

print("Failed predictions: " + str(fail_counter))

Failed predictions: 1936
